## Streaming larger-than-memory datasets
By the end of this lecture you will be able to:
- process larger-than-memory datasets with streaming
- identify which parts of a query run in the streaming engine
- run a query on the GPU engine

With the streaming engine Polars processes the query in multiple batches rather than all at once.

In [ ]:
import polars as pl

Obviously it doesn't work for me to provide very large datasets with this course. Instead we will do streaming on a small dataset and you can then apply it to your own larger datasets

In [ ]:
csv_file = "../data/nyc_trip_data_1k.csv"

We start with a simple non-streaming query where we get the average of the float columns by number of passengers

In [ ]:
(
    pl.scan_csv(csv_file,try_parse_dates = True)
    .group_by("passenger_count")
    .agg(
        pl.col(pl.Float64).mean()
    )
    .collect()
)

We ask Polars to process this query using the streaming engine by passing `engine="streaming"` to `collect`

In [ ]:
(
    pl.scan_csv(csv_file,try_parse_dates = True)
    .group_by("passenger_count")
    .agg(
        pl.col(pl.Float64).mean()
    )    
    .collect(engine="streaming")
)

With the streaming engine Polars processes the query in batches. At present only a subset of methods and expressions are supported by the streaming engine. If your full query is not supported by the streaming engine then Polars reverts back to the normal in-memory engine.

In the old streaming engine you could check if a query would run in streaming mode because it would say `STREAMING` in the query plan like this:
```
STREAMING:
  AGGREGATE
    [col("trip_distance").mean(), col("fare_amount").mean(), col("tip_amount").mean()] BY [col("passenger_count")]
    FROM
    simple π 4/4 ["trip_distance", "fare_amount", ... 2 other columns]
      Csv SCAN [../data/nyc_trip_data_1k.csv]
      PROJECT 4/7 COLUMNS
```

However, this has not yet been implemented for the new streaming engine. The plan is to have something similar for the new engine so I leave the code below but commented out.

<!-- You can use the `explain` method to see if a query will use the streaming engine by passing the `streaming=True` argument -->

In [ ]:
# print(
#     pl.scan_csv(csv_file,try_parse_dates = True)
#     .group_by("passenger_count")
#     .agg(
#         pl.col(pl.Float64).mean()
#     )
#     .explain(streaming=True)
# )

The part of the query below `STREAMING` will be executed with the streaming engine

On the other hand only the first part of this query runs with the streaming engine as `rolling` is not a supported operation in the current streaming engine

In [ ]:
# print(
#     pl.scan_csv(csv_file,try_parse_dates = True)
#     .sort("pickup")
#     .rolling(
#         index_column="pickup",
#         period="1d")
#     .agg(
#         pl.col("passenger_count").mean()
#     )
#     .explain(streaming=True)
# )

The good news is that many core methods work with the streaming engine including:
- `select`
- `with_columns`
- `filter`
- `join`
- `group_by`

<!-- However, the bad news is that even where the core method works in the streaming engine an expression that you use inside the method may not work in streaming and so that whole part of the query plan will not work in streaming.

For example, here we call `shift` within `select`. As `shift` is not currently supported by the streaming engine the entire `select` does not work in the streaming engine -->

In [ ]:
# print(
#     pl.scan_csv(csv_file,try_parse_dates = True)
#     .select(
#         pl.col("pickup").shift(1)
#     )
#     .explain(streaming=True)
# )

The reason for this is that streaming works in batches of rows. For the `shift` expression the last row in the batch needs data from another batch. This inter-batch communicaton is not currently supported and so streaming does not work for `shift`.

### Tips for working with the streaming engine

On help forums I have seen new users be frustrated with the streaming engine when they have developed a complex query which is not supported by the streaming engine and then find that they run out of memory. To avoid this I suggest following this process when processing larger-than-memory datasets:

- start off with a simple query
- check your query plan works in streaming mode with `explain`
- add additional logic to the query plan
- check your updated query plan works in streaming mode
- continue adding logic and confirming that your query plan still works in streaming mode

If you find that you have an operation that is not supported by streaming:
- see if you can reduce the size of your dataset before the non-streaming part of the query plan e.g. by `filtering` the data or doing a `group_by` aggregation
- replace non-streaming operations with streaming operations if possible

### Profiling
We can profile a query when we use streaming. Sadly this is currenrtly less useful than the non-streaming profile as the streaming part of the query plan is gathered together into a single block. Hopefully, this will be broken out in more detail in the revised streaming engine.

> If you have not encountered `profile` see the lecture on Lazy Groupby in the section on Statistics, Counts and Grouping for an introduction

In [ ]:
groupDf, profileDF = (
    pl.scan_csv(csv_file)
    .group_by("passenger_count")
    .agg(
        pl.col("trip_distance").mean()
    )
    .sort("passenger_count")
    .profile(engine="streaming",show_plot=True)
)

## GPU engine
Polars has added support for partial processing of queries on an NVIDIA GPU. If you have an NVIDIA GPU available you can run a lazy query on the GPU engine by passing `engine="gpu"` to `collect`. This is commented here as it raises an `Exception` if there is no GPU available.

In [ ]:
# print(
#     pl.scan_csv(csv_file,try_parse_dates = True)
#     .select(
#         pl.col("pickup").shift(1)
#     )
#     .collect(engine="gpu")
# )

The GPU engine is still in its early days and so only a subset of operations are supported. 

> Only NVIDIA GPUs are supported - in fact the GPU engine only happened because of support from NVIDIA. There are no plans for other GPUs to be supported.